In [215]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import matplotlib.pyplot as plt
from cassandra.cluster import Cluster
import requests
from fuzzywuzzy import fuzz
from collections import defaultdict
from fuzzywuzzy import process
from nltk.tokenize import word_tokenize

Connect to Cassandra

In [142]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

Create keyspace

In [47]:
keyspace_name = "data_quran"  


session.execute("CREATE KEYSPACE IF NOT EXISTS data_quran WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}")

session.set_keyspace('data_quran')

use keyspace

In [48]:
session.set_keyspace('data_quran')

create table

In [134]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS arabic (
        verse_number int PRIMARY KEY,
        surah int,
        ayah_id int,
        text varchar
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'arabic' created successfully!")
except Exception as e:
    print("Error:", e)

Table 'arabic' created successfully!


In [138]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS Tafseer (
        Name varchar,
        Surah int,
        Ayat int,
        Verse text,
        Tafseer text,
        PRIMARY KEY (Name, Surah, Ayat)
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'Tafseer' created successfully!")
except Exception as e:
    print("Error: Issue creating table")
    print(e)

Table 'Tafseer' created successfully!


Extract data from csv

In [135]:
# Path to your CSV file
csv_file = r"C:\Users\ASUS\Desktop\big-data-engineering-with-python-2023\final_project\data\arabic_ayah.csv"  # Replace with your CSV file path

# Read data from CSV and insert into Cassandra
try:
    # Read CSV file into DataFrame
    df = pd.read_csv(csv_file)
    
    # Iterate over DataFrame rows and insert into Cassandra
    for row in df.itertuples(index=False):
        # Prepare and execute INSERT query
        insert_query = session.prepare("INSERT INTO arabic (verse_number, surah, ayah_id, text) VALUES (?, ?, ?, ?)")
        session.execute(insert_query, (row.verse_number, row.surah, row.ayah_id, row.text))
        
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Rows")
    print(e)


Data inserted successfully!


In [140]:



file_path = r"C:\Users\ASUS\Desktop\big-data-engineering-with-python-2023\final_project\data\Quran_English_with_Tafseer.csv"

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        next(file) 
        for line in file:
            parts = line.strip().split(',')  # Assuming data is CSV
            name = parts[0]
            surah = int(parts[1])
            ayat = int(parts[2])
            verse = parts[3]
            tafseer = parts[4]
            # Prepare and execute INSERT query
            insert_query = session.prepare("INSERT INTO Tafseer (Name, Surah, Ayat, Verse, Tafseer) VALUES (?, ?, ?, ?, ?)")
            session.execute(insert_query, (name, surah, ayat, verse, tafseer))
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Data")
    print(e)

# Close connection
cluster.shutdown()


Data inserted successfully!


Show Al-quran data 

In [136]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "arabic"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")


for row in rows:
    print(row)  # 

Row(verse_number=4317, ayah_id=45, surah=42, text='وَتَرَىٰهُمۡ يُعۡرَضُونَ عَلَيۡهَا خَٰشِعِينَ مِنَ ٱلذُّلِّ يَنظُرُونَ مِن طَرۡفٍ خَفِيّٖۗ وَقَالَ ٱلَّذِينَ ءَامَنُوٓاْ إِنَّ ٱلۡخَٰسِرِينَ ٱلَّذِينَ خَسِرُوٓاْ أَنفُسَهُمۡ وَأَهۡلِيهِمۡ يَوۡمَ ٱلۡقِيَٰمَةِۗ أَلَآ إِنَّ ٱلظَّـٰلِمِينَ فِي عَذَابٖ مُّقِيمٖ')


In [144]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "Tafseer"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")


for row in rows:
    print(row)  # 

Row(name='The Reality', surah=69, ayat=1, tafseer='The Reality the Resurrection in which is realised the truth of all that was rejected in the way of the raising from the graves the reckoning and the requital or it means the Resurrection which will manifest all of that.', verse='The Reality!')


The Question

1

In [161]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="الٓمٓ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in the database.")

The phrase 'الٓمٓ' occurs 8 times in the database.


2

In [167]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="وَ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in the database.")

The phrase 'وَ' occurs 4758 times in the database.


3

In [172]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="ٱلَّذِينَ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in the database.")

The phrase 'ٱلَّذِينَ' occurs 879 times in the database.


4

In [207]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the target word
target_word = "your_target_word_here"

# Calculate similarity scores for words
word_similarity_scores = defaultdict(float)
for row in rows:
    words = word_tokenize(row.tafseer)
    for word in words:
        similarity_score = process.extractOne(target_word, [word])
        word_similarity_scores[word] = max(word_similarity_scores[word], similarity_score[1])

# Sort the words based on similarity score
similar_words = sorted(word_similarity_scores.items(), key=lambda x: x[1], reverse=True)

# Print the most similar words
print(f"The most similar words to '{target_word}' in the Tafseer text are:")
for word, similarity_score in similar_words[:10]:  # Print top 10 similar words
    print(f"- {word} (Similarity Score: {similarity_score})")

The most similar words to 'your_target_word_here' in the Tafseer text are:
- you (Similarity Score: 90)
- Our (Similarity Score: 90)
- your (Similarity Score: 90)
- Your (Similarity Score: 90)
- here (Similarity Score: 90)
- You (Similarity Score: 90)
- our (Similarity Score: 90)
- Get (Similarity Score: 90)
- her (Similarity Score: 90)
- word (Similarity Score: 90)


5

In [ ]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "battle"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'battle' appears 6 times in the 'Tafseer' table.


6

In [165]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "humans"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'humans' appears 12 times in the 'Tafseer' table.


7

In [166]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "Muslim"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'Muslim' appears 12 times in the 'Tafseer' table.


8

In [163]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "battle"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'battle' appears 6 times in the 'Tafseer' table.


9

In [182]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "peace"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'peace' appears 28 times in the 'Tafseer' table.


10

In [188]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "fira"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'fira' appears 2 times in the 'Tafseer' table.


11

In [189]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "pasture"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'pasture' appears 2 times in the 'Tafseer' table.


12

In [191]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = " obeying"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word ' obeying' appears 2 times in the 'Tafseer' table.


13

In [193]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "drunkenness"


occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'drunkenness' appears 1 times in the 'Tafseer' table.


14

In [194]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_word = "gambling"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'gambling' appears 1 times in the 'Tafseer' table.


15

In [198]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "patience"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'patience' appears 9 times in the 'Tafseer' table.


16

In [200]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "prophet"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'prophet' appears 51 times in the 'Tafseer' table.


17

In [218]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "worship"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'worship' appears 114 times in the 'Tafseer' table.


18

In [226]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "hell"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'hell' appears 31 times in the 'Tafseer' table.


19

In [228]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "fire"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'fire' appears 40 times in the 'Tafseer' table.


20

In [229]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "light"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'light' appears 50 times in the 'Tafseer' table.
